**To transfer data from surfsara to dropbox**
- select the path where the data is


In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run_super import (
    run_parallel_transfer,
    directory_scratch,
    directory_project,
    directory_archive,
    run_parallel_transfer_to_archive,
)
import dropbox
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# directory_origin = '/projects/0/einf914/agg/'
directory_origin = directory_project
# directory_origin = "/scratch-local/cbisot/"

- get the folders at this path and select the folders of interest

In [23]:
update_plate_info(directory_origin, local=True)
all_folders = get_current_folders(directory_origin, local=True)
folders = all_folders.loc[all_folders["Plate"] != "931"]

analysed:   0%|          | 0/3149 [00:00<?, ?it/s]

In [24]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime


- use the 'toward_drop.py' function including the folders of interest and using dir_drop as a parameter indicating where in dropbox the data should go
- The delete parameter indicates wether the data should be deleted from the path of origin after succesfull upload on dropbox

In [20]:
dir_drop = "DATA/PRINCE"
delete = True
run_parallel_transfer(
    "toward_drop.py",
    [dir_drop, delete],
    folders,
    1,
    "30:00",
    "staging",
    cpus=1,
    node="staging",
)
clear_output(wait=False)

**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [2]:
all_folders_drop = get_dropbox_folders("/DATA/PRINCE", True)

2022-06-08 17:08:57,355-[INFO]- dropbox:388 -> Refreshing access token.
2022-06-08 17:08:57,596-[INFO]- dropbox:477 -> Request to files/list_folder
2022-06-08 17:08:58,769-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:08:59,684-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:01,240-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:02,275-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:03,840-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:05,528-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:06,467-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:07,553-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:08,547-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-06-08 17:09:09,793-[INFO]- dropbox:477 -> Request to files/list_folder/continue
20

In [3]:
# folders_drop = all_folders_drop.loc[all_folders_drop['CrossDate'].between("20181015","20201015")]
folders_drop = all_folders_drop.loc[all_folders_drop["Plate"] == "931"]

In [6]:
folders_drop

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
7600,20220411_1910_Plate05,NaN,2022-05-20 19:33:17,data/prince/931_20220402/20220411_1910_plate05,931,20220402,931_20220402
7601,20220416_1136_Plate05,NaN,2022-05-20 19:29:14,data/prince/931_20220402/20220416_1136_plate05,931,20220402,931_20220402
7602,20220419_1143_Plate05,NaN,2022-05-20 19:29:12,data/prince/931_20220402/20220419_1143_plate05,931,20220402,931_20220402
7603,20220417_2139_Plate05,NaN,2022-05-20 19:24:59,data/prince/931_20220402/20220417_2139_plate05,931,20220402,931_20220402
7604,20220426_0110_Plate05,NaN,2022-05-20 19:24:57,data/prince/931_20220402/20220426_0110_plate05,931,20220402,931_20220402
...,...,...,...,...,...,...,...
7839,20220418_1339_Plate05,NaN,2022-05-20 09:40:22,data/prince/931_20220402/20220418_1339_plate05,931,20220402,931_20220402
7840,20220411_2310_Plate05,NaN,2022-05-20 09:38:03,data/prince/931_20220402/20220411_2310_plate05,931,20220402,931_20220402
7841,20220412_0910_Plate05,NaN,2022-05-20 09:33:44,data/prince/931_20220402/20220412_0910_plate05,931,20220402,931_20220402
7842,20220428_1728_Plate22,NaN,2022-05-20 09:31:46,data/prince/931_20220402/20220428_1728_plate22,931,20220402,931_20220402


- select the path where the folders should go

In [4]:
# directory_targ = '/projects/0/einf914/agg/'
directory_targ = directory_project
# directory_targ = "/scratch-local/cbisot/"

- use the 'from_drop.py' function including the folders of interest

In [5]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "30:00",
    "staging",
    cpus=1,
    node="staging",
)
clear_output(wait=False)

**To transfer data from dropbox to archive**
- use the 'from_drop.py' function including the folders of interest


In [4]:
folders = all_folders.loc[all_folders["Plate"].isin([792, 94])]

In [48]:
len(folders)

1858

In [47]:
unzip = "False"
flatten = False
dir_drop = "prince_data"

directory_targ = os.path.join(directory_scratch, "temp") + "/"
directory = directory_targ
run_parallel_transfer(
    "from_drop.py",
    [directory_targ, dir_drop, unzip, flatten],
    folders,
    5,
    "10:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="archiving.sh",
)
run_parallel_transfer_to_archive(
    folders, directory, "2:00:00", "staging", name_job="archiving.sh"
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418130
Submitted batch job 418131
Submitted batch job 418132
Submitted batch job 418133


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418134
Submitted batch job 418135
Submitted batch job 418136
Submitted batch job 418138


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

...

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418535
Submitted batch job 418536
Submitted batch job 418537
Submitted batch job 418538


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

Submitted batch job 418539
Submitted batch job 418540
Submitted batch job 418541
Submitted batch job 418542
Submitted batch job 418543
Submitted batch job 418544
Submitted batch job 418545
Submitted batch job 418546


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.03125 node. A full node consists of 32 CPU cores, 240000 M

In [6]:
run_parallel_transfer_to_archive(
    folders, directory, "10:00", "staging", name_job="archiving_no_wait.sh"
)

sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 374733
Submitted batch job 374734
Submitted batch job 374735
Submitted batch job 374736
Submitted batch job 374737
Submitted batch job 374738
Submitted batch job 374739
Submitted batch job 374740


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 374741
Submitted batch job 374742
Submitted batch job 374743
Submitted batch job 374744


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


Submitted batch job 374745
Submitted batch job 374746
Submitted batch job 374747
Submitted batch job 374748
Submitted batch job 374749
Submitted batch job 374750
Submitted batch job 374751
Submitted batch job 374752


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [12]:
must_zip = True
must_unzip = False
run_parallel_transfer(
    "within_surf.py",
    [directory, directory_targ, must_zip, must_unzip],
    folders,
    5,
    "10:00",
    "staging",
    cpus=1,
    node="staging",
)

Submitted batch job 242022


sbatch: Single node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.


In [51]:
def get_dropbox_folders_new(dir_drop: str) -> pd.DataFrame:
    data = []
    dbx = load_dbx()
    response = dbx.files_list_folder(dir_drop, recursive=True)
    # for fil in response.entries:
    listfiles = []
    listjson = []
    while response.has_more:
        listfiles += [
            file for file in response.entries if file.name.split("/")[-1] == "param.m"
        ]
        response = dbx.files_list_folder_continue(response.cursor)
    listfiles += [
        file for file in response.entries if file.name.split("/")[-1] == "param.m"
    ]
    # print([((file.path_lower.split(".")[0]) + "_info.json") for file in listfiles if (file.name.split(".")[-1] == "zip") &
    #        (((file.path_lower.split(".")[0]) + "_info.json") not in listjson)])
    listfiles.reverse()
    names = [file.path_lower.split("/")[-2] for file in listfiles]
    path_drop = [os.path.join(*file.path_lower.split("/")[:-1]) for file in listfiles]
    id_uniques = [path.split("/")[-2] for path in path_drop]
    plate_num = [idi.split("_")[0] for idi in id_uniques]
    date_cross = [idi.split("_")[1] for idi in id_uniques]
    sizes = [file.size / 10**9 for file in listfiles]
    modified = [file.client_modified for file in listfiles]
    df = pd.DataFrame(
        (names, sizes, modified, path_drop, plate_num, date_cross, id_uniques)
    ).transpose()
    df = df.rename(
        columns={
            0: "folder",
            1: "size",
            2: "change_date",
            3: "tot_path_drop",
            4: "Plate",
            5: "CrossDate",
            6: "unique_id",
        }
    )
    return df

In [52]:
all_folders_drop_new = get_dropbox_folders_new("/DATA/PRINCE")

2022-05-18 12:32:45,427-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-18 12:32:45,577-[INFO]- dropbox:477 -> Request to files/list_folder
2022-05-18 12:32:46,459-[INFO]- dropbox:477 -> Request to files/list_folder/continue
2022-05-18 12:32:47,417-[INFO]- dropbox:477 -> Request to files/list_folder/continue


In [53]:
all_folders_drop_new

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
0,20201204_2123_plate39,4.1e-07,2022-05-17 09:03:06,data/prince/94_20201123/20201204_2123_plate39,94,20201123,94_20201123
1,20201204_1723_plate39,4.1e-07,2022-05-17 09:03:04,data/prince/94_20201123/20201204_1723_plate39,94,20201123,94_20201123
2,20201205_0123_plate39,4.1e-07,2022-05-17 09:03:03,data/prince/94_20201123/20201205_0123_plate39,94,20201123,94_20201123
3,20201204_1323_plate39,4.1e-07,2022-05-17 09:03:02,data/prince/94_20201123/20201204_1323_plate39,94,20201123,94_20201123
4,20201205_0523_plate39,4.1e-07,2022-05-17 09:03:02,data/prince/94_20201123/20201205_0523_plate39,94,20201123,94_20201123
...,...,...,...,...,...,...,...
164,20201123_2200_plate39,4.1e-07,2022-05-16 15:49:31,data/prince/94_20201123/20201123_2200_plate39,94,20201123,94_20201123
165,20201123_1601_plate39,4.1e-07,2022-05-16 15:49:29,data/prince/94_20201123/20201123_1601_plate39,94,20201123,94_20201123
166,20201123_1701_plate39,4.1e-07,2022-05-16 15:49:28,data/prince/94_20201123/20201123_1701_plate39,94,20201123,94_20201123
167,20201123_1400_plate39,4.1e-07,2022-05-16 15:49:27,data/prince/94_20201123/20201123_1400_plate39,94,20201123,94_20201123


In [60]:
directory_targ2 = os.path.join(directory_scratch, "temptemp") + "/"

In [106]:
dbx = load_dbx()

path = "/" + all_folders_drop_new.iloc[0]["tot_path_drop"]
response = dbx.files_list_folder(path, recursive=False)
listfiles = []
listjson = []
while response.has_more:
    listfiles += [file for file in response.entries]
    response = dbx.files_list_folder_continue(response.cursor)
listfiles += [file for file in response.entries]

2022-05-18 14:42:40,500-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-18 14:42:40,675-[INFO]- dropbox:477 -> Request to files/list_folder


In [107]:
listfiles

[FileMetadata(client_modified=datetime.datetime(2022, 5, 17, 9, 3, 6), content_hash='c51bd427fc91a8869dd28bd0453ceac525009c1b9c7ee3bfae79278b539ec129', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:tRFPXS1OOAQAAAAAAAC7wg', is_downloadable=True, media_info=NOT_SET, name='param.m', parent_shared_folder_id='10269534512', path_display='/DATA/PRINCE/94_20201123/20201204_2123_Plate39/param.m', path_lower='/data/prince/94_20201123/20201204_2123_plate39/param.m', property_groups=NOT_SET, rev='015df3168bbf6cc00000002641ca930', server_modified=datetime.datetime(2022, 5, 17, 9, 3, 7), sharing_info=FileSharingInfo(modified_by='dbid:AABIDXJq597-uS2e5SgTFZ0vARb3NPh-G8s', parent_shared_folder_id='10269534512', read_only=False), size=410, symlink_info=NOT_SET),
 FileMetadata(client_modified=datetime.datetime(2022, 5, 17, 9, 9, 25), content_hash='28e3d41e0b306f8d053bb4ee8d5964d4aa13c64b1fd67df360f691617289cffd', export_info=NOT_SET, file_lock_info=NOT_SET, has

In [109]:
[file.path_lower for file in listfiles]

['/data/prince/94_20201123/20201204_2123_plate39/param.m',
 '/data/prince/94_20201123/20201204_2123_plate39/stitchedimage.tif',
 '/data/prince/94_20201123/20201204_2123_plate39/princemeasurement.csv',
 '/data/prince/94_20201123/20201204_2123_plate39/analysis.zip',
 '/data/prince/94_20201123/20201204_2123_plate39/img.zip']

In [113]:
def download_folders_drop(folders_drop: pd.DataFrame, directory_target):
    dbx = load_dbx()
    for index, row in folders_drop.iterrows():
        path = "/" + row["tot_path_drop"]
        response = dbx.files_list_folder(path, recursive=False)
        listfiles = []
        while response.has_more:
            listfiles += [file for file in response.entries]
            response = dbx.files_list_folder_continue(response.cursor)
        listfiles += [file for file in response.entries]
        folder = row["folder"]
        path_folder = os.path.join(directory_target, folder)
        if not os.path.exists(path_folder):
            os.mkdir(path_folder)
        for file in listfiles:
            path_drop = file.path_lower
            path_local = os.path.join(
                directory_target, folder, path_drop.split("/")[-1]
            )
            print(path_drop, path_local)

            download(path_drop, path_local, unzip=(path_drop[-4:] == ".zip"))

In [114]:
download_folders_drop(all_folders_drop_new, directory_targ2)

2022-05-18 14:46:27,627-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-18 14:46:27,775-[INFO]- dropbox:477 -> Request to files/list_folder
/data/prince/94_20201123/20201204_2123_plate39/param.m /scratch-shared/amftrack/temptemp/20201204_2123_plate39/param.m
2022-05-18 14:46:28,158-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-18 14:46:28,358-[INFO]- dropbox:477 -> Request to files/download
/data/prince/94_20201123/20201204_2123_plate39/stitchedimage.tif /scratch-shared/amftrack/temptemp/20201204_2123_plate39/stitchedimage.tif
2022-05-18 14:46:29,835-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-18 14:46:29,987-[INFO]- dropbox:477 -> Request to files/download
/data/prince/94_20201123/20201204_2123_plate39/princemeasurement.csv /scratch-shared/amftrack/temptemp/20201204_2123_plate39/princemeasurement.csv
2022-05-18 14:46:30,741-[INFO]- dropbox:388 -> Refreshing access token.
2022-05-18 14:46:30,899-[INFO]- dropbox:477 -> Request to files/download
/data/prince

KeyboardInterrupt: 

In [8]:
import os
import sys


# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    update_plate_info,
    get_current_folders,
)

from datetime import datetime, timedelta
from amftrack.util.dbx import upload_folders
from time import time_ns
directory_origin = r"/mnt/sun-temp/TEMP/PRINCE_syncing/"
dir_drop = "DATA/PRINCE"
suffix_data_info = str(time_ns())
update_plate_info(directory_origin, local=True,strong_constraint=False,suffix_data_info = suffix_data_info)
all_folders_origin = get_current_folders(directory_origin, local=True)

all_folders_origin["date_datetime"] = pd.to_datetime(
    all_folders_origin["date"].astype(str), format="%d.%m.%Y, %H:%M:"
)
selection = (datetime.now() - all_folders_origin["date_datetime"]) <= timedelta(days=3)
current_prince = all_folders_origin.loc[selection]
plates_in_prince = current_prince['unique_id'].unique()
old_folders = all_folders_origin.loc[all_folders_origin['unique_id'].isin(plates_in_prince)==False]

old_folders["Plate"] = (
    old_folders["Plate"].str.replace("R", "66666").str.replace("[^0-9]", "")
)


analysed:   0%|          | 0/7379 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [6]:
old_folders = old_folders.loc[old_folders['PrincePos'].astype(int)%2==0]

In [7]:
old_folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,unique_id,datetime,date_datetime
13,/mnt/sun-temp/TEMP/PRINCE_syncing/20220615_131...,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,"15.06.2022, 13:13:",20220615_1313_Plate12,30_20220509,2022-06-15 13:13:00,2022-06-15 13:13:00
15,/mnt/sun-temp/TEMP/PRINCE_syncing/20220617_110...,22,10,'Carrot','Agg','001P100N100C','Y','-',20220522,25,...,False,False,False,False,False,"17.06.2022, 11:04:",20220617_1104_Plate10,22_20220522,2022-06-17 11:04:00,2022-06-17 11:04:00
27,/mnt/sun-temp/TEMP/PRINCE_syncing/20220531_233...,22,10,'Carrot','Agg','001P100N100C','Y','-',20220522,25,...,False,False,False,False,False,"31.05.2022, 23:37:",20220531_2337_Plate10,22_20220522,2022-05-31 23:37:00,2022-05-31 23:37:00
35,/mnt/sun-temp/TEMP/PRINCE_syncing/20220524_221...,22,10,'Carrot','Agg','001P100N100C','Y','-',20220522,25,...,False,False,False,False,False,"24.05.2022, 22:16:",20220524_2216_Plate10,22_20220522,2022-05-24 22:16:00,2022-05-24 22:16:00
52,/mnt/sun-temp/TEMP/PRINCE_syncing/20220602_194...,30,12,'Carrot','Agg','001P100N200C','Y','-',20220509,25,...,False,False,False,False,False,"02.06.2022, 19:47:",20220602_1947_Plate12,30_20220509,2022-06-02 19:47:00,2022-06-02 19:47:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7453,/mnt/sun-temp/TEMP/PRINCE_syncing/20220526_023...,1040,22,'Carrot','A5','001P100N100C','Y','-',20220502,25,...,False,False,False,False,False,"26.05.2022, 02:36:",20220526_0236_Plate22,1040_20220502,2022-05-26 02:36:00,2022-05-26 02:36:00
7461,/mnt/sun-temp/TEMP/PRINCE_syncing/20220605_215...,1039,18,'Carrot','C2','001P100N200C','Y','-',20220516,25,...,False,False,False,False,False,"05.06.2022, 21:52:",20220605_2152_Plate18,1039_20220516,2022-06-05 21:52:00,2022-06-05 21:52:00
7463,/mnt/sun-temp/TEMP/PRINCE_syncing/20220521_054...,1039,18,'Carrot','C2','001P100N200C','Y','-',20220516,25,...,False,False,False,False,False,"21.05.2022, 05:40:",20220521_0540_Plate18,1039_20220516,2022-05-21 05:40:00,2022-05-21 05:40:00
7466,/mnt/sun-temp/TEMP/PRINCE_syncing/20220530_192...,1039,18,'Carrot','C2','001P100N200C','Y','-',20220516,25,...,False,False,False,False,False,"30.05.2022, 19:20:",20220530_1920_Plate18,1039_20220516,2022-05-30 19:20:00,2022-05-30 19:20:00
